In [4]:
# Simple Imports

import torch 
from torch.utils.data import DataLoader
from evaluate import load

from accelerate import Accelerator, DistributedType
from datasets import load_dataset
from transformers import (
    AdamW,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    set_seed,
)

from tqdm.auto import tqdm

import datasets
import transformers

In [57]:
model_checkpoint = "huawei-noah/TinyBERT_General_4L_312D"

In [8]:
raw_dataset = load_dataset('glue',"mrpc")

In [9]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [15]:
raw_dataset['train'][1]

{'sentence1': "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
 'sentence2': "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
 'label': 0,
 'idx': 1}

In [41]:
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    picks = random.sample(range(len(dataset)), num_examples)  # Ensures unique picks
    df = pd.DataFrame(dataset[picks])

    for column, feature in dataset.features.items():
        if isinstance(feature, datasets.ClassLabel):
            df[column] = df[column].apply(lambda i: feature.names[i])  # Use apply instead of transform

    display(HTML(df.to_html()))

show_random_elements(raw_dataset["train"])


,sentence1,sentence2,label,idx
0,"The shares represent more than half the 115.9 million shares Turner held at the end of April , according to Bloomberg data .",The sale represents about 52 per cent of the 115.9 million shares Mr Turner held at the end of April .,equivalent,2861
1,""" We have some small opportunities in November and maybe January , "" Mr. Parsons said optimistically .",""" I think we have some opportunities -- some small opportunities -- in November and possibly January , "" he said .",equivalent,823
2,"Stony Brook University launched the study in 1996 , after earlier studies indicated a possible connection between electromagnetic fields and cancer .","The State University at Stony Brook launched the study in 1996 , after earlier studies indicated a possible connection .",equivalent,2790
3,"Hackett and Rossignol did not know each other and Hackett had no connection to Colby , Doyle said .","State police Lt. Timothy Doyle said Hackett and Rossignol did not know each other , and that Hackett had no connection to the college .",equivalent,3191
4,"It 's almost as if they ( Russians ) hit an x-mark on the ground , "" NASA spokesman Robert Navias said .","It 's almost as if they ( Russians ) hit an x-mark on the ground . """,equivalent,2703


In [42]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoing)

config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [45]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.","this is the third one")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [101, 2023, 2003, 1996, 2353, 2028, 102]}

In [46]:
def tokenizer_fuction(examples):
    outputs = tokenizer(examples["sentence1"], examples["sentence2"], truncation= True, padding="max_length", max_length=128)
    return outputs

In [ ]:
tokenizer_fuction(raw_dataset["train"][:1])

{'input_ids': [[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1,

In [50]:
tokenized_datasets = raw_dataset.map(tokenizer_fuction, batched=True, remove_columns=["idx", "sentence1", "sentence2"])


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [55]:
tokenized_datasets['train'].features

{'labels': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [54]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [56]:
tokenized_datasets.set_format("torch")

In [58]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)


pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

In [59]:
def create_dataloaders(train_batch_size = 8, eval_batch_size = 32):
    train_dataloader = DataLoader(
        tokenized_datasets["train"],shuffle=True, batch_size=train_batch_size
    )
    eval_dataloader = DataLoader(
        tokenized_datasets["validation"],shuffle=False,batch_size=eval_batch_size
    )
    return train_dataloader, eval_dataloader

In [ ]:
train_dataloader, eval_dataloader = create_dataloaders()

In [ ]:
batch = next(iter(train_dataloader))
print(batch)

{'labels': tensor([1, 0, 1, 1, 1, 1, 1, 1]), 'input_ids': tensor([[ 101, 2302, 1996,  ...,    0,    0,    0],
        [ 101, 4614, 2056,  ...,    0,    0,    0],
        [ 101, 5747, 1005,  ...,    0,    0,    0],
        ...,
        [ 101, 1996, 8431,  ...,    0,    0,    0],
        [ 101, 2016, 2056,  ...,    0,    0,    0],
        [ 101, 1996, 2142,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [65]:

for batch in train_dataloader:
    print({k: v.shape for k, v in batch.items()})
    outputs = model(**batch)
    break

{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 128]), 'token_type_ids': torch.Size([8, 128]), 'attention_mask': torch.Size([8, 128])}


In [66]:
outputs

SequenceClassifierOutput(loss=tensor(0.6899, grad_fn=<NllLossBackward0>), logits=tensor([[ 3.1039e-03,  3.1664e-03],
        [ 5.8471e-03,  9.7055e-03],
        [-4.2013e-03,  1.4204e-02],
        [-9.0647e-05,  1.2822e-02],
        [ 8.0254e-03,  1.1794e-02],
        [ 4.0683e-03,  2.1559e-02],
        [ 4.4362e-03,  1.4851e-02],
        [-4.6327e-03,  1.6866e-02]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [67]:
from evaluate import load

# Load the metric using the new `evaluate` library
metric = load("glue", "mrpc")

# Example predictions and labels
predictions = outputs.logits.detach().argmax(dim=-1)  # Convert logits to predicted class
references = batch["labels"]  # Ground truth labels

# Compute the metric
results = metric.compute(predictions=predictions, references=references)

print(results)  # {'accuracy': 0.875, 'f1': 0.9333}


{'accuracy': 0.5, 'f1': 0.6666666666666666}


In [68]:
hyperparameters = {
    "learning_rate": 2e-5,
    "num_epochs": 3,
    "train_batch_size": 8, # Actual batch size will this x 8
    "eval_batch_size": 32, # Actual batch size will this x 8
    "seed": 42,
}

In [69]:
def training_function(model):
    # Initialize accelerator
    accelerator = Accelerator()
    
    # To have only one message (and not 8) per logs of Transformers or Datasets, we set the logging verbosity
    # to INFO for the main process only.
    if accelerator.is_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()
        
    train_dataloader, eval_dataloader = create_dataloaders(
        train_batch_size=hyperparameters["train_batch_size"],
        eval_batch_size=hyperparameters['eval_batch_size'],
    )
    
    set_seed(hyperparameters["seed"])
    
    # Instantiate optimizer
    optimizer = AdamW(params=model.parameters(), lr=hyperparameters["learning_rate"])
    
    # Prepare everything
    # There is no specific order to remember, we just need to unpack the objects in the same order we gave them to the
    # prepare method.
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader    
    )
    
    num_epochs = hyperparameters['num_epochs']
    
    # Instantiate learning rate scheduler after preparing the training dataloader as the prepare method
    # may change its length.
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer = optimizer,
        num_warmup_steps= 100,
        num_training_steps= len(train_dataloader) * num_epochs,
    )
    
    # Instantiate a progress bar to keep track of training. Note that we only enable it on the main
    # process to avoid having 8 progress bars.
    progress_bar = tqdm(range(num_epochs * len(train_dataloader)), disable=not accelerator.is_main_process)
    
    for epoch in range(num_epochs):
        model.train()
        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

        model.eval()
        all_predictions = []
        all_labels = []

        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                outputs = model(**batch)
            predictions = outputs.logits.argmax(dim=-1)

            # We gather predictions and labels from the 8 TPUs to have them all.
            all_predictions.append(accelerator.gather(predictions))
            all_labels.append(accelerator.gather(batch["labels"]))

        # Concatenate all predictions and labels.
        # The last thing we need to do is to truncate the predictions and labels we concatenated
        # together as the prepared evaluation dataloader has a little bit more elements to make
        # batches of the same size on each process.
        all_predictions = torch.cat(all_predictions)[:len(tokenized_datasets["validation"])]
        all_labels = torch.cat(all_labels)[:len(tokenized_datasets["validation"])]

        eval_metric = metric.compute(predictions=all_predictions, references=all_labels)

        # Use accelerator.print to print only on the main process.
        accelerator.print(f"epoch {epoch}:", eval_metric)
    

In [71]:
from accelerate import notebook_launcher

notebook_launcher(training_function, (model,),num_processes=1)

Launching training on one GPU.


/home/shahidmo/miniconda3/envs/nlp_quant/lib/python3.11/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

epoch 0: {'accuracy': 0.7990196078431373, 'f1': 0.8586206896551725}
epoch 1: {'accuracy': 0.8088235294117647, 'f1': 0.8733766233766234}
epoch 2: {'accuracy': 0.8284313725490197, 'f1': 0.8805460750853242}
